In [624]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to change the path if needed.)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read the School Data and Student Data and store into a Pandas DataFrame
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

# Check names. Grades not shown for individual students to maintain FERPA rules.
student_data_df["student_name"].head(10)

0       Paul Bradley
1       Victor Smith
2    Kevin Rodriguez
3      Richard Scott
4         Bonnie Ray
5      Bryan Miranda
6      Sheena Carter
7       Nicole Baker
8       Michael Roth
9     Matthew Greene
Name: student_name, dtype: object

## Deliverable 1: Replace the reading and math scores.

### Replace the 9th grade reading and math scores at Thomas High School with NaN.

In [633]:
# Install numpy using conda install numpy or pip install numpy. 
# Step 1. Import numpy as np.
import numpy as np

In [634]:
# Step 2. Use the loc method on the student_data_df to select all the reading scores from the 9th grade at Thomas High School and replace them with NaN.
student_data_df.loc[(student_data_df["school_name"]=="Thomas High School") & (student_data_df["grade"]=="9th"),"reading_score"] = np.nan


In [635]:
#  Step 3. Refactor the code in Step 2 to replace the math scores with NaN.
student_data_df.loc[(student_data_df["school_name"]=="Thomas High School") & (student_data_df["grade"]=="9th"),"math_score"] = np.nan
Thomas_9th_df = student_data_df.loc[(student_data_df["school_name"]=="Thomas High School") & (student_data_df["grade"]=="9th")]


In [636]:
#  Step 4. Check the student data for NaN's. 
# Students names are hidden to maintain FERPA rules.
student_data_df[["grade","school_name","reading_score","math_score"]].tail(10)

,grade,school_name,reading_score,math_score
39160,11th,Thomas High School,89.0,86.0
39161,10th,Thomas High School,70.0,84.0
39162,12th,Thomas High School,94.0,77.0
39163,11th,Thomas High School,90.0,75.0
39164,9th,Thomas High School,NaN,NaN
39165,12th,Thomas High School,99.0,90.0
39166,10th,Thomas High School,95.0,70.0
39167,9th,Thomas High School,NaN,NaN
39168,10th,Thomas High School,99.0,90.0
39169,11th,Thomas High School,95.0,75.0


## Deliverable 2 : Repeat the school district analysis

### District Summary

In [640]:
# Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
# Students names are hidden to maintain FERPA rules.
school_data_complete_df[["grade","school_name","reading_score","math_score","School ID","type","size","budget"]].tail()

,grade,school_name,reading_score,math_score,School ID,type,size,budget
39165,12th,Thomas High School,99.0,90.0,14,Charter,1635,1043130
39166,10th,Thomas High School,95.0,70.0,14,Charter,1635,1043130
39167,9th,Thomas High School,NaN,NaN,14,Charter,1635,1043130
39168,10th,Thomas High School,99.0,90.0,14,Charter,1635,1043130
39169,11th,Thomas High School,95.0,75.0,14,Charter,1635,1043130


In [570]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()

# Calculate the Total Budget
total_budget = school_data_df["budget"].sum()

In [571]:
# Calculate the Average Scores using the "clean_student_data".
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [572]:
# Step 1. Get the number of students that are in ninth grade at Thomas High School.
# These students have no grades. 
Thomas_9th_df_count = Thomas_9th_df["Student ID"].count()

# Get the total student count 
student_count = school_data_complete_df["Student ID"].count()

# Step 2. Subtract the number of students that are in ninth grade at 
# Thomas High School from the total student count to get the new total student count.
new_total_student_count = student_count - Thomas_9th_df_count

In [573]:
# Calculate the passing rates using the "clean_student_data".
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]

In [574]:
# Step 3. Calculate the passing percentages with the new total student count.
passing_math_percentage = passing_math_count / new_total_student_count * 100
passing_reading_percentage = passing_reading_count / new_total_student_count * 100

In [575]:
# Calculate the students who passed both reading and math.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)
                                               & (school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students that passed both reading and math.
overall_passing_math_reading_count = passing_math_reading["student_name"].count()

# Step 4.Calculate the overall passing percentage with new total student count.
overall_passing_percentage = overall_passing_math_reading_count / new_total_student_count * 100

In [576]:
# Create a DataFrame
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count, 
          "Total Students": student_count, 
          "Total Budget": total_budget,
          "Average Math Score": average_math_score, 
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage,
          }])

# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format the "Total Budget" to have the comma for a thousands separator, a decimal separator and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

# Display the data frame
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.9,81.9,74.8,85.7,64.9


##  School Summary

In [577]:
# Determine the School Type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [578]:
# Create the DataFrame
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})

# per_school_summary_df.head()

In [579]:
# Format the Total School Budget and the Per Student Budget
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


In [580]:
# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).
THSstudent_10to12_df = school_data_complete_df[(school_data_complete_df["school_name"] == "Thomas High School")
                                               & (school_data_complete_df["grade"] != "9th")]

In [581]:
# Step 6. Get all the students passing math from THS
THSstudent_10to12_passMath_count = THSstudent_10to12_df[(THSstudent_10to12_df["math_score"] >= 70)].count()

In [582]:
# Step 7. Get all the students passing reading from THS
THSstudent_10to12_passRead_count = THSstudent_10to12_df[(THSstudent_10to12_df["reading_score"] >= 70)].count()

In [583]:
# Step 8. Get all the students passing math and reading from THS
THS_10to12_passMathRead_count = THSstudent_10to12_df[(THSstudent_10to12_df["reading_score"] >= 70) 
                                                     & (THSstudent_10to12_df["math_score"] >= 70)].count()

In [584]:
# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School. 
THSstudent_10to12_passmath_percentage = THSstudent_10to12_passMath_count / THSstudent_10to12_count *100

In [585]:
# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.
THSstudent_10to12_passread_percentage = THSstudent_10to12_passRead_count / THSstudent_10to12_count *100

In [586]:
# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 
THS_Overall_PassMathRead_percentage = THS_10to12_passMathRead_count / THSstudent_10to12_count *100

In [587]:
# Step 12. Replace the passing math percent for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School","% Passing Math"] = THSstudent_10to12_passmath_percentage["math_score"]

In [588]:
# Step 13. Replace the passing reading percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School","% Passing Reading"] = THSstudent_10to12_passread_percentage["reading_score"]

In [589]:
# Step 14. Replace the overall passing percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School","% Overall Passing"] = THS_Overall_PassMathRead_percentage["grade"]

In [590]:
per_school_summary_df.tail()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.350937,83.896082,93.185690,97.018739,90.630324
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,93.333333,96.611111,90.333333


## High and Low Performing Schools 

In [591]:
# Sort and show top five schools.
per_school_summary_df.sort_values(by="% Overall Passing", ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.350937,83.896082,93.185690,97.018739,90.630324
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [592]:
# Sort and show bottom five schools.
per_school_summary_df.sort_values(by="% Overall Passing", ascending=True).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math and Reading Scores by Grade

In [593]:
# Create a Series of scores by grade levels using conditionals.
Grade9_df = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
Grade10_df = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
Grade11_df = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
Grade12_df = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

# Group each school Series by the school name for the average math score.
# The average math score for each grade level from each school (3 pt)
Grade9_AverageMath = Grade9_df.groupby(["school_name"]).mean()["math_score"]
Grade10_AverageMath = Grade10_df.groupby(["school_name"]).mean()["math_score"]
Grade11_AverageMath = Grade11_df.groupby(["school_name"]).mean()["math_score"]
Grade12_AverageMath = Grade12_df.groupby(["school_name"]).mean()["math_score"]

# # Group each school Series by the school name for the average reading score.
# # The average reading score for each grade level from each school (3 pt)
Grade9_AverageRead = Grade9_df.groupby(["school_name"]).mean()["reading_score"]
Grade10_AverageRead = Grade10_df.groupby(["school_name"]).mean()["reading_score"]
Grade11_AverageRead = Grade11_df.groupby(["school_name"]).mean()["reading_score"]
Grade12_AverageRead = Grade12_df.groupby(["school_name"]).mean()["reading_score"]

In [594]:
# Combine each Series for average math scores by school into single data frame.
AverageMath_Grade = pd.DataFrame({              
               "9th": Grade9_AverageMath,
               "10th": Grade10_AverageMath,
               "11th": Grade11_AverageMath,
               "12th": Grade12_AverageMath})
# AverageMath_Grade

In [595]:
# Combine each Series for average reading scores by school into single data frame.
AverageRead_Grade = pd.DataFrame({              
               "9th": Grade9_AverageRead,
               "10th": Grade10_AverageRead,
               "11th": Grade11_AverageRead,
               "12th": Grade12_AverageRead})
# AverageRead_Grade

In [596]:
# Format each grade column.
AverageMath_Grade["9th"] = AverageMath_Grade["9th"].map("{:.1f}".format)
AverageMath_Grade["10th"] = AverageMath_Grade["10th"].map("{:.1f}".format)
AverageMath_Grade["11th"] = AverageMath_Grade["11th"].map("{:.1f}".format)
AverageMath_Grade["12th"] = AverageMath_Grade["12th"].map("{:.1f}".format)

AverageRead_Grade["9th"] = AverageRead_Grade["9th"].map("{:.1f}".format)
AverageRead_Grade["10th"] = AverageRead_Grade["10th"].map("{:.1f}".format)
AverageRead_Grade["11th"] = AverageRead_Grade["11th"].map("{:.1f}".format)
AverageRead_Grade["12th"] = AverageRead_Grade["12th"].map("{:.1f}".format)


In [597]:
# Remove the index.
AverageMath_Grade.index.name = None

# Display the data frame
AverageMath_Grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [598]:
# Remove the index.
AverageRead_Grade.index.name = None

# Display the data frame
AverageRead_Grade

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


## Scores by School Spending

In [599]:
# Establish the spending bins and group names.
# per_school_capita.describe()  #min=578, std=28.5, max=655
bins_spend = [0, 585, 625, 640, 680]
group_names = ["<$585", "$586-625", "$626-640", "$641-680"]
# Cut the per_school_capita into the spending ranges.
per_school_capita.groupby(pd.cut(per_school_capita, bins_spend)).count()
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, bins_spend, labels=group_names)
# per_school_summary_df.head()

In [600]:
# Calculate averages for the desired columns. 
spend_ScoresMath = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spend_ScoresRead = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spend_PassMath = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spend_PassRead = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_SpendPassing = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]
# overall_SpendPassing

In [601]:
# Create the DataFrame
spend_Summary_df = pd.DataFrame({
          "Average Math Score" : spend_ScoresMath,
          "Average Reading Score": spend_ScoresRead,
          "% Passing Math": spend_PassMath,
          "% Passing Reading": spend_PassRead,
          "% Overall Passing": overall_SpendPassing})

# spend_Summary_df

In [602]:
# Format the DataFrame 
spend_Summary_df["Average Math Score"] = spend_Summary_df["Average Math Score"].map("{:.1f}".format)
spend_Summary_df["Average Reading Score"] = spend_Summary_df["Average Reading Score"].map("{:.1f}".format)
spend_Summary_df["% Passing Math"] = spend_Summary_df["% Passing Math"].map("{:.1f}".format)
spend_Summary_df["% Passing Reading"] = spend_Summary_df["% Passing Reading"].map("{:.1f}".format)
spend_Summary_df["% Overall Passing"] = spend_Summary_df["% Overall Passing"].map("{:.1f}".format)
spend_Summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.5,83.9,93.5,96.6,90.4
$586-625,83.5,83.9,94.0,96.3,90.3
$626-640,78.5,81.7,73.1,85.0,62.9
$641-680,77.0,81.0,66.7,80.7,53.7


## Scores by School Size

In [617]:
# Establish the bins.
# per_school_counts.describe()  #min=427, std=1421, max=4976
bins_size=[0, 1000, 2000, 3000, 4000, 5000]
group_names = ["<1,000", "1,001-2,000", "2,001-3,000", "3,001-4,000", "4,001-5,000"]

# Cut the per_school_counts into the size ranges.
per_school_counts.groupby(pd.cut(per_school_counts, bins_size)).count()
per_school_summary_df["Scores (Per School Size)"] = pd.cut(per_school_counts, bins_size, labels=group_names)

# per_school_summary_df

In [618]:
# Calculate averages for the desired columns. 
size_ScoresMath = per_school_summary_df.groupby(["Scores (Per School Size)"]).mean()["Average Math Score"]
size_ScoresRead = per_school_summary_df.groupby(["Scores (Per School Size)"]).mean()["Average Reading Score"]
size_PassMath = per_school_summary_df.groupby(["Scores (Per School Size)"]).mean()["% Passing Math"]
size_PassRead = per_school_summary_df.groupby(["Scores (Per School Size)"]).mean()["% Passing Reading"]
overall_SizePassing = per_school_summary_df.groupby(["Scores (Per School Size)"]).mean()["% Overall Passing"]
# overall_SizePassing

In [619]:
# Assemble into DataFrame. 
size_Summary_df = pd.DataFrame({
          "Average Math Score" : size_ScoresMath,
          "Average Reading Score": size_ScoresRead,
          "% Passing Math": size_PassMath,
          "% Passing Reading": size_PassRead,
          "% Overall Passing": overall_SizePassing})

# size_Summary_df

In [620]:
# Format the DataFrame  
size_Summary_df["Average Math Score"] = size_Summary_df["Average Math Score"].map("{:.1f}".format)
size_Summary_df["Average Reading Score"] = size_Summary_df["Average Reading Score"].map("{:.1f}".format)
size_Summary_df["% Passing Math"] = size_Summary_df["% Passing Math"].map("{:.1f}".format)
size_Summary_df["% Passing Reading"] = size_Summary_df["% Passing Reading"].map("{:.1f}".format)
size_Summary_df["% Overall Passing"] = size_Summary_df["% Overall Passing"].map("{:.1f}".format)
size_Summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Scores (Per School Size),,,,,
"<1,000",83.8,83.9,93.6,96.1,89.9
"1,001-2,000",83.4,83.9,93.6,96.7,90.6
"2,001-3,000",78.4,81.8,73.5,84.5,62.9
"3,001-4,000",76.8,80.7,66.4,80.2,53.0
"4,001-5,000",77.1,81.0,66.5,81.3,53.9


## Scores by School Type

In [621]:
# Calculate averages for the desired columns. 
# per_school_types = school_data_complete_df["type"]
# per_school_types.describe()  # unique=2
# per_school_types.unique()   # array(['District', 'Charter'], dtype=object)

type_ScoresMath = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
type_ScoresRead = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
type_PassMath = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_PassRead = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
overall_TypePassing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]
# overall_TypePassing

In [622]:
# Assemble into DataFrame. 
type_Summary_df = pd.DataFrame({
          "Average Math Score" : type_ScoresMath,
          "Average Reading Score": type_ScoresRead,
          "% Passing Math": type_PassMath,
          "% Passing Reading": type_PassRead,
          "% Overall Passing": overall_TypePassing})
# type_Summary_df

In [623]:
# # Format the DataFrame 
type_Summary_df["Average Math Score"] = type_Summary_df["Average Math Score"].map("{:.1f}".format)
type_Summary_df["Average Reading Score"] = type_Summary_df["Average Reading Score"].map("{:.1f}".format)
type_Summary_df["% Passing Math"] = type_Summary_df["% Passing Math"].map("{:.1f}".format)
type_Summary_df["% Passing Reading"] = type_Summary_df["% Passing Reading"].map("{:.1f}".format)
type_Summary_df["% Overall Passing"] = type_Summary_df["% Overall Passing"].map("{:.1f}".format)

type_Summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,93.6,96.6,90.4
District,77.0,81.0,66.5,80.8,53.7
